In [ ]:
'''🔹 PART 1  Advanced Filtering & Conditions
 1  Filter passengers with:
Age between 18 and 40
Fare > median fare
Gender = female

 2  Select passengers who:
Are NOT in 3rd class
Survived
Embarked from "S"

 3  Count how many passengers match these filters

🔹 PART 2  GroupBy Tricks
 4  Group by Pclass and Gender and calculate:
Average Age
Survival Rate (hint: .mean() on "Survived")
 5  Find the class with the highest fare variance
 6  Use .agg() to calculate multiple stats:
For Fare: min, max, mean, std grouped by Embarked

🔹 PART 3  Pivot Tables & Custom Columns
 7  Create a pivot table:
Rows → Pclass
Columns → Gender
Values → Mean Fare

 8  Create a new column "FareLevel":
If Fare < 10 → "Low"
If Fare between 1050 → "Medium"
If Fare > 50 → "High"

 9  Use .apply() to label "AgeGroup" as:
012 → Child
1319 → Teen
2059 → Adult
60+ → Senior

 10  Create a summary table:
Count of each AgeGroup by Gender and Survival status'''

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

In [ ]:
median_fare = df['Fare'].median()
mask = (df['Age'] < 41) & (df['Age'] > 17) & (df['Fare'] > median_fare) & (df['Sex'] == 'female')
filtered_df = df[mask]
#print(filtered_df.sample(n = 5))
print(filtered_df['PassengerId'].count())

In [ ]:
mask = (df['Pclass'] != 3) & (df['Survived'] == 1) & (df['Embarked'] == 'S')
selected_passengers = df[mask]
#print(selected_passengers.sample(n = 5))
print(selected_passengers['PassengerId'].count())

In [ ]:
df = df.rename(columns = {'Sex' : 'Gender'})
avg_age = df.groupby(['Pclass','Gender'])['Age'].mean()
surviv_rate = df.groupby(['Pclass','Gender'])['Survived'].mean()
print(avg_age, surviv_rate)

In [ ]:
fare_var_by_class = df.groupby('Pclass')['Fare'].var()
class_indx = fare_var_by_class.idxmax()
print(class_indx)

In [ ]:
fare_stats = df.groupby('Embarked')['Fare'].agg(
    min_fare = 'min',
    max_fare = 'max',
    mean_fare = 'mean',
    std_fare = 'std'
)
print(fare_stats)

In [ ]:
pivot_class_gender = df.pivot_table(values = 'Fare', index = 'Pclass', columns = 'Gender', aggfunc = 'mean')
print(pivot_class_gender)

In [ ]:
import numpy as np
conditions = [
    df['Fare'] < 10,
    (df['Fare'] >= 10) & (df['Fare'] <= 50),
    df['Fare'] > 50
]
choices = ['Low','Medium','High']
df['FareLevel'] = np.select(conditions,choices,default = 'Unknown')
print(df['FareLevel'].sample(n = 5))

In [ ]:
df['AgeGroup'] = df['Age'].apply(lambda age : 'Child' if age < 13 else
                                        'Unknown'if age == pd.isna(age) else
                                        'Teen' if age < 20 else
                                        'Adult' if age < 60 else
                                        'Senior')
print(df['AgeGroup'].head())

In [26]:
table_group = df.groupby(['AgeGroup','Gender','Survived']).size().unstack(fill_value = 0)
print(table_group)

Survived           0    1
AgeGroup Gender          
Adult    female   42  140
         male    278   64
Child    female   13   19
         male     16   21
Senior   female   17   40
         male    127   19
Teen     female    9   34
         male     47    5
